<a href="https://colab.research.google.com/github/TAM-DS/Quantum-Hybrid-Moons-Classifier/blob/main/Hybrid_QNN_Moons_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

!pip install qiskit qiskit-machine_learning

# ── Qiskit imports ────────────────────────────────────────
from qiskit.circuit.library import RealAmplitudes
from qiskit.primitives import StatevectorEstimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.quantum_info import SparsePauliOp # Import SparsePauliOp

# Set seeds for reproducibility
torch.manual_seed(42)

# ── 1. Data ────────────────────────────────────────────────
X, y = make_moons(n_samples=100, noise=0.1, random_state=42)
schaler = StandardScaler()
X_scaled = schaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test,  dtype=torch.long)

# ── 2. Quantum Circuit ─────────────────────────────────────
num_qubits = 2 # Changed from 1 to 2 to match the output of the classical layer
qc = RealAmplitudes(num_qubits, reps=3)

# ── 3. EstimatorQNN ────────────────────────────────────────
estimator = StatevectorEstimator()
# Define two observables for two outputs (e.g., expectation value of Pauli Z on each qubit)
observables = [SparsePauliOp('ZI'), SparsePauliOp('IZ')]
qnn = EstimatorQNN(
    circuit=qc,
    estimator=estimator,
    input_params=qc.parameters[:num_qubits],     # data encoding
    weight_params=qc.parameters[num_qubits:],    # trainable weights
    observables=observables # Specify observables to get 2 outputs
)

# ── 4. Torch Connector ─────────────────────────────────────
quantum_layer = TorchConnector(qnn)

# ── 5. Hybrid Model ────────────────────────────────────────
class HybridQuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)          # classical pre-processing
        self.quantum = quantum_layer        # quantum layer (outputs 2 values)
        # No final linear needed — quantum already gives 2-dim output

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.quantum(x)                 # shape: (batch, 2)
        return x

# ── 6. Model, Loss, Optimizer ──────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HybridQuantumClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

X_train, y_train = X_train.to(device), y_train.to(device)
X_test,  y_test  = X_test.to(device),  y_test.to(device)

# ── 7. Training Loop ───────────────────────────────────────
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# ── 8. Evaluation ──────────────────────────────────────────
model.eval()
with torch.no_grad():
    y_pred = model(X_test).argmax(dim=1)
    accuracy = (y_pred == y_test).float().mean().item()
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

/tmp/ipython-input-3952146113.py:33: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  qc = RealAmplitudes(num_qubits, reps=3)


Epoch [10/50], Loss: 0.5693
Epoch [20/50], Loss: 0.5455
Epoch [30/50], Loss: 0.5382
Epoch [40/50], Loss: 0.5304
Epoch [50/50], Loss: 0.5220
Test Accuracy: 100.00%
